In [1]:
%matplotlib inline
# %load ../../utility/initialization.py
'''
How to Train Your Dragon: V4
Sequentially initialize FT reactive distillation model automatically
'''
# system imports
import sys
import os
import datetime
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# import pickle
import dill
from copy import deepcopy

# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import beautify, beautify_reactive, HiddenLogs, HiddenPrints, plot_distribution
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF, check_iteration
from utility.model_utility import which_MPCC, select_MPCC, augmented_objective, add_solver
from utility.time_utility import create_filename_time, log_now, log_end

2018-07-26 03:15:44 - Start Program


In [2]:
logname = create_filename_time()
log_text_dir = './log/text/opt_'+logname+'.dat'
log_figure_dir = './log/figure/opt_'+logname+'.pdf'

In [3]:
with open('./log/model/stage_20_pf.pickle','rb') as f:
    model = dill.load(f)

In [4]:
# for j in model.reactive:
#     model.reactive[j].MPCC_P_pf.rho = 10000
# model.reboiler.MPCC_P_pf.rho = 10000

In [5]:
for j in model.reactive:
    model.reactive[j].MPCC_P_NCP.epi = 1e-7
model.reboiler.MPCC_P_NCP.epi = 1e-7

In [6]:
opt = add_solver(pe, max_iter = 500, warm_start = True, output = True, scale = True)

In [7]:
for j in model.reactive:
    select_MPCC(model.reactive[j],'NCP')
select_MPCC(model.reboiler,'NCP')

> Selected MPCC: reactive[1].MPCC_P_NCP
s_L:  1.964812775101591e-08
s_V:  0.0

> Selected MPCC: reactive[2].MPCC_P_NCP
s_L:  2.282828746889007e-08
s_V:  0.0

> Selected MPCC: reactive[3].MPCC_P_NCP
s_L:  2.541409097349865e-08
s_V:  0.0

> Selected MPCC: reactive[4].MPCC_P_NCP
s_L:  2.8964348141935536e-08
s_V:  0.0

> Selected MPCC: reactive[5].MPCC_P_NCP
s_L:  3.6801820770708585e-08
s_V:  0.0

> Selected MPCC: reactive[6].MPCC_P_NCP
s_L:  5.067497122028571e-08
s_V:  0.0

> Selected MPCC: reactive[7].MPCC_P_NCP
s_L:  8.886416478990899e-08
s_V:  0.0

> Selected MPCC: reactive[8].MPCC_P_NCP
s_L:  2.636894172205893e-07
s_V:  0.0

> Selected MPCC: reactive[9].MPCC_P_NCP
s_L:  2.915820537690447e-07
s_V:  0.0

> Selected MPCC: reactive[10].MPCC_P_NCP
s_L:  2.557694373298898e-07
s_V:  0.0

> Selected MPCC: reactive[11].MPCC_P_NCP
s_L:  2.931749515530684e-07
s_V:  0.0

> Selected MPCC: reactive[12].MPCC_P_NCP
s_L:  2.215560288433305e-07
s_V:  0.0

> Selected MPCC: reactive[13].MPCC_P_NCP
s_L:  

In [8]:
model.del_component(model.obj)
model.obj = augmented_objective(pe,model,expr = sum(model.reactive[j].T for j in model.TRAY_reactive), sense = pe.maximize)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> reactive[8].T + reactive[9].T + reactive[10].T + reactive[11].T + reactive[12].T + reactive[14].T + reactive[15].T + reactive[16].T + reactive[17].T + reactive[18].T
------------------------------------------------------------------------------------------------------------


In [9]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling=mc19
max_iter=500
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
output_file=./tmp/ipopt_output_tmp.output
linear_scaling_on_demand=no


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 500                   yes
                                 mu_init = 1e-06                 yes
                             output_file = ./tmp/ipopt_output_tmp.output  yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
     

In [10]:
# pdf = PdfPages(log_figure_dir)

In [11]:
with HiddenLogs(log_text_dir,'w'):
    print('\n>','Original 20 stage case')
    print('-'*108)
    beautify(pe,model)
    log_now()

# plot_distribution(model,pdf,'Original 20 stage case')


> Original 20 stage case
------------------------------------------------------------------------------------------------------------
Here comes the result:
Total Conversion: 81.20%
------------------------------------------------------------------------------------------------------------
stages       T      Q                                            V               L       P            W     
condenser    30.00  -134.                                        3.0267          0.6407  0.0337       2.0654

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
NON--[1]     119.3  0.000       0.000  0.000  0.000  0.000       5.7667  0.0000  0.7437  0.0000       20.000
NON--[2]     131.1  0.000       0.000  0.000  0.000  0.000       5.8697  0.0000  0.6752  0.0000       20.000
NON--[3]     140.3  0.000       0.000  0.000  0.000  0.000       5.8012  0.0000  0.6136  0.0000       20.000
NON--[4]     148.7  0.000       0.000  0.000  0.000  

# Optimization Input

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw ratio
    * PR_L: 0 - 1

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
**Objective:**
* Max gasoline production

### Parameters

In [12]:
for j in model.TRAY_reactive:
    model.reactive[j].T.setlb(200+273.15)
    model.reactive[j].T.setub(300+273.15)
    
model.condenser.PR_L.unfix()
model.condenser.PR_L.setlb(0.05)
model.condenser.PR_L.setub(0.5)

In [13]:
model.reactive[4].PR_L.unfix()
model.reactive[7].PR_L.unfix()
model.reactive[10].PR_L.unfix()

### Constraints

In [14]:
model.quality_coefficient = pe.Param(within=pe.NonNegativeReals,mutable=True,initialize=1)

In [15]:
model.product_spec_con = pe.ConstraintList()

In [16]:
model.product_spec_con.add(expr = sum(model.condenser.x[i] for i in m.PRODUCT_cnumber['naphtha']) >= \
                           model.quality_coefficient*0.75*sum(model.condenser.x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in m.PRODUCT_cnumber['gasoline']) >= \
                           model.quality_coefficient*0.75*sum(model.reactive[7].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[10].x[i] for i in m.PRODUCT_cnumber['diesel']) >= \
                           model.quality_coefficient*0.6*sum(model.reactive[10].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reboiler.x[i] for i in m.PRODUCT_cnumber['heavy']) >= \
                           model.quality_coefficient*0.85*sum(model.reboiler.x[i] for i in m.COMP_ORG));

### Objective

In [17]:
model.del_component(model.obj)
model.obj = augmented_objective(pe,model,expr = model.reactive[7].L['P'], sense = pe.maximize)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> reactive[7].L[P]
------------------------------------------------------------------------------------------------------------


In [18]:
# itr_count1 = 0
# meta_results = {}
# while True:
#     itr_count1 += 1
#     opt = add_solver(pe, max_iter = itr_count1, warm_start = True, output = True, scale = True)
#     model_tmp = deepcopy(model)
#     results = opt.solve(model_tmp,tee=False)
#     model_tmp.solutions.store_to(results)
#     meta_results[itr_count1] = results
#     print('itr_count',itr_count1,'obj',model.obj())
#     if results.solver.termination_condition.key == 'optimal':
#         break

In [19]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [20]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Optimized Product Side Draw')
#     print('-'*108)
#     beautify(pe,model)
#     # check_product_spec(model)
#     log_now()

# # plot_distribution(model,pdf,'Optimized Product Side Draw')

# Open up feed allocation

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [21]:
for j in model.reactive:
    model.reactive[j].F.unfix()
    model.reactive[j].F.setlb(0)
    model.reactive[j].F.setub(10)

In [22]:
model.total_feed_con = pe.ConstraintList()

In [23]:
model.total_feed_con.add(expr = sum(model.reactive[j].F for j in model.reactive) == 10);

In [24]:
model.reactive[14].F.setlb(1)

In [25]:
itr_count1 = 0
meta_results = {}
while True:
    itr_count1 += 1
    opt = add_solver(pe, max_iter = itr_count1, warm_start = True, output = True, scale = True)
    model_tmp = deepcopy(model)
    results = opt.solve(model_tmp,tee=False)
    model_tmp.solutions.store_to(results)
    meta_results[itr_count1] = results
    print('itr_count',itr_count1,'obj',model_tmp.obj())
    if results.solver.termination_condition.key == 'optimal':
        break

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 1 obj 0.0389716082457106
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 2 obj 0.05202387038812374
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 3 obj 0.06421548485352736
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 4 obj 0.06772686313358582
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 5 obj 0.07009438785329315
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 6 obj 0.07650729632116109
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 7 obj 0.08045106142533659
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 8 obj 0.08263314004995133
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 9 obj 0.08

itr_count 40 obj 0.10426704236551369
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 41 obj 0.10455215963242892
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 42 obj 0.10587772628153466
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 43 obj 0.10658553399819456
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 44 obj 0.10779572282805122
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 45 obj 0.10892606250312319
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 46 obj 0.11018476049087607
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 47 obj 0.11028745321297388
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 48 obj 0.11269786402605718
	message from solver=Ipopt 3.12.8\x3a Maximum Numbe

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 80 obj 0.11621793771449246
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 81 obj 0.11615038793309745
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 82 obj 0.11622753053937045
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 83 obj 0.11624222214613049
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 84 obj 0.11624301574905613
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 85 obj 0.11624471980618482
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 86 obj 0.1162456920913156
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 87 obj 0.11626233800688139
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 88

In [26]:
with open('./meta_results_NCP_1step_F14.pickle','wb') as f:
    dill.dump(meta_results,f)

In [27]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling=mc19
max_iter=99
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
output_file=./tmp/ipopt_output_tmp.output
linear_scaling_on_demand=no


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 99                    yes
                                 mu_init = 1e-06                 yes
                             output_file = ./tmp/ipopt_output_tmp.output  yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
      

  66 -1.1597351e-01 1.41e+01 9.80e+01  -6.0 5.51e+03 -11.3 2.18e-01 6.50e-02h  1
  67 -1.1597935e-01 1.84e+01 3.27e+02  -6.0 6.01e+03    -  4.05e-01 1.18e-01h  1
  68 -1.1598858e-01 3.70e+00 1.54e+03  -6.0 2.08e+02    -  2.04e-01 8.21e-01h  1
  69 -1.1599198e-01 2.66e+00 9.39e+02  -6.0 4.81e+02    -  3.39e-01 3.88e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70 -1.1600336e-01 1.15e+00 4.03e+02  -6.0 7.92e+01    -  8.29e-01 5.70e-01h  1
  71 -1.1603317e-01 2.04e-01 2.57e+01  -6.0 1.32e+02    -  8.61e-01 1.00e+00h  1
  72 -1.1601292e-01 1.04e-01 1.42e+01  -6.0 9.42e+01    -  1.00e+00 1.00e+00h  1
  73 -1.1600787e-01 1.03e-03 5.60e-01  -6.0 6.22e+00    -  1.00e+00 1.00e+00h  1
  74 -1.1600818e-01 2.56e-05 6.64e-04  -6.0 1.48e+00    -  1.00e+00 1.00e+00h  1
  75 -1.1600820e-01 1.32e-07 1.60e-05  -6.0 1.06e-01    -  1.00e+00 1.00e+00h  1
  76 -1.1600820e-01 2.86e-10 1.52e-11  -6.0 2.46e-05    -  1.00e+00 1.00e+00h  1
  77 -1.1609830e-01 1.17e-01

In [28]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Feed Amount and Location')
    print('-'*108)
    beautify(pe,model)
    # check_product_spec(model)
    log_now()

# plot_distribution(model,pdf,'Optimized Feed Amount and Location')


> Optimized Feed Amount and Location
------------------------------------------------------------------------------------------------------------
Here comes the result:
Total Conversion: 86.08%
------------------------------------------------------------------------------------------------------------
stages       T      Q                                            V               L       P            W     
condenser    30.00  -140.                                        2.3281          0.6306  0.0877       2.2018

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
NON--[1]     123.0  0.000       0.000  0.000  0.000  0.000       5.2483  0.0000  0.7493  0.0000       20.000
NON--[2]     133.4  0.000       0.000  0.000  0.000  0.000       5.3670  0.0000  0.6985  0.0000       20.000
NON--[3]     140.9  0.000       0.000  0.000  0.000  0.000       5.3161  0.0000  0.6581  0.0000       20.000
NON--[4]     147.4  0.000       0.000  0.

# Increase spec a little bit

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= **0.8**
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [29]:
# model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in product['gasoline']) >= \
#                            model.quality_coefficient*0.8*sum(model.reactive[7].x[i] for i in m.COMP_ORG));

In [30]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [31]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Increase Gasoline Specification to 80%')
#     print('-'*108)
#     beautify(pe,model)
#     check_product_spec(model)
#     log_now()

# plot_distribution(model,pdf,'Increase Gasoline Specification to 80%')

# Open up catalyst allocation ( = TOTAL reconstruction, long wait time)

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3
        
* Catalyst
    * cat: 0 - 10000

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75 / 0.8
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * sum feed = 10 kmol/s
    
* Total catalyst
    * sum cat = 30000 kg
    
**Objective:**
* Max gasoline production

In [32]:
# for j in model.reactive:
#     model.reactive[j].cat.unfix()
#     model.reactive[j].cat.setlb(0)
#     model.reactive[j].cat.setub(30000)

In [33]:
# model.total_cat_con = pe.ConstraintList()

In [34]:
# model.total_cat_con.add(expr = sum(model.reactive[j].cat for j in model.reactive) == 10*3000);

In [35]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [36]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Optimized Catalyst Amount and Location')
#     print('-'*108)
#     beautify(pe,model)
#     check_product_spec(model)
#     log_now()

# plot_distribution(model,pdf,'Optimized Catalyst Amount and Location')

In [37]:
# pdf.close()

In [38]:
# with open('./log/model/{}.pickle'.format(logname),'wb') as f:
#     dill.dump(model,f)

In [39]:
# for j in model.TRAY_reactive:
#     print('stage',j,'\n')
#     print('Total Flow\t{}'.format(model.reactive[j].F.value + sum(model.reactive[j].r_total_comp[i].value for i in m.COMP_TOTAL)))
#     for i in m.COMP_TOTAL:
#         if i in m.COMP_FEED:
#             print('{}\t\t{}'.format(i,model.reactive[j].F.value*model.reactive[j].z[i].value + model.reactive[j].r_total_comp[i].value))
#         else:
#             print('{}\t\t{}'.format(i,model.reactive[j].r_total_comp[i].value))